# urgent satd characterization process

## 1.随机从 output_file_gai.csv 文件中，分别抽取 50 条 surive_time=0 和 surive_time=1 的数据，且保证 function_before_tokenized 和 function_after_tokenized 字段不为空。

In [1]:
import pandas as pd
import random

# 读取CSV文件
output_df = pd.read_csv('output_file_gai.csv')

# 过滤掉 `function_before_tokenized` 和 `function_after_tokenized` 为空的行
filtered_df = output_df[(output_df['function_before_tokenized'].notna()) & (output_df['function_after_tokenized'].notna())]

# 分别提取 `surive_time=0` 和 `surive_time=1` 的数据
surive_time_0 = filtered_df[filtered_df['survive_time'] == 0]
surive_time_1 = filtered_df[filtered_df['survive_time'] >= 1]

# 随机抽取50条数据
sample_0 = surive_time_0.sample(n=50, random_state=42)  # random_state 是为了保证可重复性
sample_1 = surive_time_1.sample(n=50, random_state=42)

# 保存抽取的数据到文件（可选）
sample_0.to_csv('sample_0.csv', index=False)
sample_1.to_csv('sample_1.csv', index=False)

print("Sample extraction complete.")

Sample extraction complete.


## 2.提取特征
<img src="特征维度.png" width=1200>

- Churn Workload：改动的代码量，根据token化之后比较function_before and function_after的相似度，然后显著性检验 
- Commit Goal: SATD注释是否清晰，目标明确
- Working on release: 离上一次发布的版本的时间
- Project Startup: 离项目刚开始的时间
- Commit workload: 提交次数，代表开发人员做出的不同贡献的数量
- Tenure：作者关于这个项目的经验

### Churn Workload : 改动的代码量，根据token化之后比较function_before and function_after的相似度，然后显著性检验

- 读取CSV文件：从 sample_0.csv 和 sample_1.csv 文件中读取数据。
- 相似度计算函数：使用 difflib.SequenceMatcher 函数来计算两个字符串的相似度。该函数返回值介于 0 到 1 之间，1 表示完全相同，0 表示完全不同。
- 应用相似度计算：使用 apply() 方法逐行应用相似度计算，并将结果存储到新的 similarity 列中。
- 保存结果：将包含相似度的结果保存到新的 CSV 文件中（sample_0_with_similarity.csv 和 sample_1_with_similarity.csv）。

In [2]:
import pandas as pd
import difflib

# 读取CSV文件
sample_0 = pd.read_csv('sample_0.csv')
sample_1 = pd.read_csv('sample_1.csv')

# 定义计算相似度的函数
def calculate_similarity(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).ratio()

# 计算 sample_0 的相似度
sample_0['similarity'] = sample_0.apply(lambda row: calculate_similarity(row['function_before_tokenized'], row['function_after_tokenized']), axis=1)

# 计算 sample_1 的相似度
sample_1['similarity'] = sample_1.apply(lambda row: calculate_similarity(row['function_before_tokenized'], row['function_after_tokenized']), axis=1)

# 保存结果到新文件
sample_0.to_csv('sample_0_with_similarity.csv', index=False)
sample_1.to_csv('sample_1_with_similarity.csv', index=False)

print("Similarity calculation complete.")

Similarity calculation complete.


- 读取 CSV 文件：使用 pandas 库读取 sample_0.csv 和 sample_1.csv 文件。
- 获取相似度数据：假设你已经计算了相似度，并将相似度存储在 similarity 列中。我们通过 dropna() 方法确保没有缺失值。
- t 检验：使用 scipy.stats.ttest_ind() 函数对 survive_time=0 和 survive_time=1 的相似度数据进行独立样本 t 检验。
- 显著性检验：根据 p 值判断是否有显著性差异，通常使用的显著性水平为 0.05。

In [15]:
from scipy import stats

# 读取 CSV 文件
sample_0 = pd.read_csv('sample_0_with_similarity.csv')
sample_1 = pd.read_csv('sample_1_with_similarity.csv')

# 假设相似度数据存储在 "similarity" 列
survive_time_0 = sample_0['similarity'].dropna()  # 确保数据中没有缺失值
survive_time_1 = sample_1['similarity'].dropna()

# 进行独立样本 t 检验
t_statistic, p_value = stats.ttest_ind(survive_time_0, survive_time_1)

# 输出结果
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# 检查显著性水平 (通常使用 alpha = 0.05)
alpha = 0.05
if p_value < alpha:
    print("两个数据集之间存在显著差异。")
else:
    print("两个数据集之间不存在显著差异。")

T-statistic: -2.1048953643854884
P-value: 0.03785823687703119
两个数据集之间存在显著差异。


> 结果：
> T-statistic: -2.1048953643854884
> P-value: 0.03785823687703119
> 两个数据集之间存在显著差异。
#### 解释：修改的部分越多，相似度越低，意味着更需要被修改或者是问题存在的比较大，更紧急，所以存在的版本更少

计算平均值和中位数

In [4]:
import pandas as pd

# 读取 CSV 文件
sample_0 = pd.read_csv('sample_0_with_similarity.csv')
sample_1 = pd.read_csv('sample_1_with_similarity.csv')

# 假设相似度数据存储在 "similarity" 列
survive_time_0 = sample_0['similarity'].dropna()  # 确保没有缺失值
survive_time_1 = sample_1['similarity'].dropna()

# 计算平均值和中位数
mean_0 = survive_time_0.mean()
median_0 = survive_time_0.median()

mean_1 = survive_time_1.mean()
median_1 = survive_time_1.median()

# 输出结果
print(f"Survive_time=0 的平均值: {mean_0}")
print(f"Survive_time=0 的中位数: {median_0}")

print(f"Survive_time=1 的平均值: {mean_1}")
print(f"Survive_time=1 的中位数: {median_1}")

Survive_time=0 的平均值: 0.6833865717347414
Survive_time=0 的中位数: 0.7542094756904894
Survive_time=1 的平均值: 0.7800271740024162
Survive_time=1 的中位数: 0.868339775028901


### Commit Goal : 注释的单词数量，注释的目的性是否明确？注释包含的信息量

#### 1. 计算单词数量
- 使用 apply(lambda x: len(str(x).split())) 计算每条 comment 的单词数量。
- 保存结果：将包含单词数量的结果保存到新的 CSV 文件中，分别为 sample_0_with_word_count.csv 和 sample_1_with_word_count.csv（这一步是可选的）。
- 统计单词数量：对两个数据集中的单词数量分别计算平均值和中位数。
- 显著性检验：通过 t 检验分析两个数据集中 comment_word_count 是否存在显著差异。

In [5]:
import pandas as pd
from scipy import stats

# 读取 CSV 文件
sample_0 = pd.read_csv('sample_0.csv')
sample_1 = pd.read_csv('sample_1.csv')

# 假设 comment 数据存储在 "comment" 列，计算单词数量
sample_0['comment_word_count'] = sample_0['comment'].apply(lambda x: len(str(x).split()))
sample_1['comment_word_count'] = sample_1['comment'].apply(lambda x: len(str(x).split()))

# 保存带有 comment_word_count 列的新文件（可选）
sample_0.to_csv('sample_0_with_word_count.csv', index=False)
sample_1.to_csv('sample_1_with_word_count.csv', index=False)

# 提取 comment_word_count 列
comment_word_count_0 = sample_0['comment_word_count'].dropna()  # 确保没有缺失值
comment_word_count_1 = sample_1['comment_word_count'].dropna()

# 计算平均值和中位数
mean_word_count_0 = comment_word_count_0.mean()
median_word_count_0 = comment_word_count_0.median()

mean_word_count_1 = comment_word_count_1.mean()
median_word_count_1 = comment_word_count_1.median()

# 输出平均值和中位数
print(f"Survive_time=0 的评论平均单词数: {mean_word_count_0}")
print(f"Survive_time=0 的评论中位数单词数: {median_word_count_0}")

print(f"Survive_time=1 的评论平均单词数: {mean_word_count_1}")
print(f"Survive_time=1 的评论中位数单词数: {median_word_count_1}")

# 进行独立样本 t 检验
t_statistic, p_value = stats.ttest_ind(comment_word_count_0, comment_word_count_1)

# 输出 t 检验结果
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

# 检查显著性水平 (通常使用 alpha = 0.05)
alpha = 0.05
if p_value < alpha:
    print("两个数据集之间存在显著差异。")
else:
    print("两个数据集之间不存在显著差异。")

Survive_time=0 的评论平均单词数: 8.06
Survive_time=0 的评论中位数单词数: 7.0
Survive_time=1 的评论平均单词数: 8.06
Survive_time=1 的评论中位数单词数: 7.0
T-statistic: 0.0
P-value: 1.0
两个数据集之间不存在显著差异。


#####  Mann-Whitney U 检验

In [18]:
from scipy.stats import mannwhitneyu

# 进行 Mann-Whitney U 检验
u_statistic, p_value_mw = mannwhitneyu(comment_word_count_0, comment_word_count_1)

# 输出结果
print(f'U-statistic: {u_statistic}')
print(f'P-value: {p_value_mw}')

# 检查显著性水平 (通常使用 alpha = 0.05)
alpha = 0.05
if p_value_mw < alpha:
    print("两个数据集之间存在显著差异。")
else:
    print("两个数据集之间不存在显著差异。")


U-statistic: 1265.0
P-value: 0.9199521670134884
两个数据集之间不存在显著差异。


#### 2. 使用LLM，星火认知大模型，`isClear()` 判别SATD的目标是否明确

In [28]:
from sparkai.llm.llm import ChatSparkLLM, ChunkPrintHandler
from sparkai.core.messages import ChatMessage
import time


def isClear(comment):
    #星火认知大模型Spark Max的URL值，其他版本大模型URL值请前往文档（https://www.xfyun.cn/doc/spark/Web.html）查看
    SPARKAI_URL = 'wss://spark-api.xf-yun.com/v4.0/chat'
    #星火认知大模型调用秘钥信息，请前往讯飞开放平台控制台（https://console.xfyun.cn/services/bm35）查看
    SPARKAI_APP_ID = 'b24d9eaf'
    SPARKAI_API_SECRET = 'NDk1ZTg0ZjY0ZTdmMTZlY2EzYWI1NDZm'
    SPARKAI_API_KEY = '026cf9486631a31bf441ad99c68ebc23'
    #星火认知大模型Spark Max的domain值，其他版本大模型domain值请前往文档（https://www.xfyun.cn/doc/spark/Web.html）查看
    SPARKAI_DOMAIN = '4.0Ultra'
    spark = ChatSparkLLM(
        spark_api_url=SPARKAI_URL,
        spark_app_id=SPARKAI_APP_ID,
        spark_api_key=SPARKAI_API_KEY,
        spark_api_secret=SPARKAI_API_SECRET,
        spark_llm_domain=SPARKAI_DOMAIN,
        streaming=False,
    )
    """
        判断SATD注释是否清晰，目标明确。
        
        Args:
        comment: SATD注释字符串。
        
        Returns:
        True or False。
    """
    messages = [ChatMessage(
        role="user",
        content=f'你是一个软件工程领域的工作者，你对SATD自承认技术债比较了解，你认为注释{comment}的目的含义是否清晰'
    )]
    handler = ChunkPrintHandler()
    time.sleep(1)  # 休眠
    answer = spark.generate([messages], callbacks=[handler]).generations[0][0].text
    print(answer,answer=="Yes")
    # return answer == "Yes"

分别对sample.csv文件中的comment进行判定

In [29]:
import pandas as pd

# 读取 sample_0.csv 和 sample_1.csv 文件
sample_0 = pd.read_csv('sample_0.csv')
sample_1 = pd.read_csv('sample_1.csv')

# 对 sample_0 的评论进行判定
sample_0['is_clear'] = sample_0['comment'].apply(isClear)

# 对 sample_1 的评论进行判定
sample_1['is_clear'] = sample_1['comment'].apply(isClear)

# 保存带有 is_clear 列的文件
sample_0.to_csv('sample_0_with_is_clear.csv', index=False)
sample_1.to_csv('sample_1_with_is_clear.csv', index=False)

print("判定结果已保存到 sample_0_with_is_clear.csv 和 sample_1_with_is_clear.csv 文件中。")


2024-10-10 19:42:39 中国标准时间 - SparkPythonSDK - ERROR - [d:\dev\python3.8.10\lib\site-packages\sparkai\llm\llm.py:694] - SparkLLMClient wait LLM api response timeout 30 seconds


作为一个软件工程领域的工作者，我认为注释TODO (jon) this is fails most of the time, and is truly disgusting的目的含义并不清晰。以下是对这句话的具体分析：

1. **代码维护的复杂性**
   - **技术债的影响**：自承认技术债务（SATD）通常指开发人员为了实现短期目标而牺牲长期代码质量的行为。这种注释可能暗示了代码中存在某种未解决的问题或临时解决方案，从而增加了未来的维护难度。
   - **可读性和可维护性**：这样的注释可能会降低代码的可读性和可维护性，使得其他开发者难以理解代码的真实意图和存在的问题。

2. **开发流程的不规范**
   - **缺乏规范的开发流程**：这样的注释可能反映了开发过程中缺乏规范的流程和标准，导致问题被随意地记录下来，而不是通过正式的问题跟踪系统进行管理。
   - **团队沟通不足**：注释中的个人标记（如“jon”）可能表明团队内部缺乏有效的沟通机制，导致问题只能通过个人注释来记录。

3. **项目压力与时间限制**
   - **项目进度的压力**：在面对紧迫的项目进度时，开发人员可能会采取快速但不完美的解决方案，并留下这样的注释作为提醒。
   - **资源和时间的限制**：资源和时间的限制也可能是导致这种情况的原因之一，开发人员在有限的时间内无法找到更合适的解决方案。

4. **个人习惯与责任感**
   - **个人编码习惯**：某些开发人员可能有在代码中留下个人标记的习惯，这可能是一种个人风格或习惯。
   - **责任感的体现**：尽管这种方式并不理想，但至少表明开发人员对自己的工作有一定的责任感，愿意在未来回来解决问题。

5. **技术挑战与不确定性**
   - **技术难题的存在**：这样的注释可能指向一个技术上的难题，开发人员暂时无法解决，只能先记录下来。
   - **不确定性的处理**：在软件开发过程中，面对不确定性，开发人员可能会采取保守的策略，先记录问题，等待更好的解决方案出现。

6. **心理因素与情绪表达**
   - **情绪宣泄**：使用“disgusting”这样的词汇可能是开发人员对当前代码状态或自己处理方式的一种情绪宣泄。
   - **心理压力的反映**：长期的工作压力和不断的技术挑战可能导致开发人员

2024-10-10 19:45:02 中国标准时间 - SparkPythonSDK - ERROR - [d:\dev\python3.8.10\lib\site-packages\sparkai\llm\llm.py:694] - SparkLLMClient wait LLM api response timeout 30 seconds


注释 `TODO: Make sure the bounds of EditRect don't exceed the workspace` 的目的含义相对清晰，但可能需要一些上下文来完全理解。以下是对这一注释的解释：

1. **EditRect**：这通常指的是一个矩形区域，用于表示用户界面中的编辑或选择区域。它可能包含起始点（x, y）和宽度、高度等属性。

2. **Bounds**：这指的是EditRect的边界，即它的左上角和右下角的坐标。这些边界定义了EditRect在屏幕上或工作区内的具体位置和大小。

3. **Workspace**：这通常指的是应用程序的工作区域或画布，用户可以在这个区域内进行各种操作。这个区域可能有固定的尺寸，或者有特定的边界限制。

4. **Make sure the bounds of EditRect don't exceed the workspace**：这句话的意思是确保EditRect的边界不要超出工作区的范围。换句话说，就是确保EditRect始终完全位于工作区内，不会部分或全部超出工作区的边界。

### 为什么这个注释是必要的？

- **用户体验**：如果EditRect超出了工作区，可能会导致用户无法看到或编辑该区域的内容，从而影响用户体验。
- **程序稳定性**：超出边界可能导致未定义的行为或错误，例如访问无效内存或触发异常。
- **功能正确性**：确保所有操作都在预期的范围内进行，避免因边界问题导致的功能失效或错误。

### 如何实现这个目标？

要实现这个目标，可以采取以下措施：

1. **边界检查**：在设置或修改EditRect的位置和大小时，添加代码以检查其边界是否超出了工作区的边界。
2. **约束条件**：在用户界面逻辑中添加约束条件，确保任何操作都不会使EditRect超出工作区。
3. **反馈机制**：如果检测到边界超出，提供适当的用户反馈，如警告消息或自动调整EditRect的位置。

通过这些措施，可以确保EditRect始终在工作区内，从而提高程序的稳定性和用户体验。 False
作为软件工程领域的工作者，我认为注释“TODO the credentials are stored somewhere else...”的目的含义是相对清晰的。以下是对该注释目的含义的详

2024-10-10 19:49:58 中国标准时间 - SparkPythonSDK - ERROR - [d:\dev\python3.8.10\lib\site-packages\sparkai\llm\llm.py:694] - SparkLLMClient wait LLM api response timeout 30 seconds


...

在软件工程中，注释是开发者用来记录代码意图、解释复杂逻辑或标记待办事项的重要工具。对于“TODO: uncommenting this line causes a hang!”这样的注释，其目的和含义相对清晰，但也存在一些潜在的问题。以下是对这类注释的详细分析：

### 一、注释的目的与含义

1. **提醒开发者注意潜在问题**：此注释的主要目的是提醒其他开发者（包括作者自己）在取消注释该行代码时可能会遇到程序挂起（hang）的问题。这是一个重要的提示，因为它可能涉及到底层资源管理、线程同步或其他复杂的系统交互问题。
2. **标记已知缺陷或待解决问题**：通过使用“TODO”标签，注释表明这里存在一个已知的问题，需要在未来的开发周期中解决。这有助于团队成员了解代码的当前状态，并优先处理这些标记的区域。
3. **防止误操作**：明确指出取消注释会导致程序挂起，可以防止开发者在不知情的情况下进行修改，从而避免引入难以调试的错误。

### 二、潜在问题与改进建议

1. **缺乏具体信息**：虽然注释指出了存在一个问题，但并没有提供足够的上下文来解释为什么取消注释会导致程序挂起。为了提高可维护性，建议在注释中添加更多细节，比如涉及的具体函数、可能的原因以及已经尝试过的解决办法。
2. **更新不及时的风险**：随着时间的推移，代码库会发生变化，如果这个注释没有被及时更新或移除，可能会导致误导。因此，建议建立一个定期审查代码注释的机制，确保所有“TODO”项都是最新的，并且在问题解决后及时删除或更新相关注释。
3. **影响代码整洁度**：过多的注释可能会降低代码的可读性，尤其是当它们分散在多个地方时。考虑使用集中的文档或专门的注释块来记录这类信息，以保持代码的整洁。

综上所述，“TODO: uncommenting this line causes a hang!”这样的注释在提醒开发者注意潜在问题方面是有效的，但为了更好地服务于软件开发过程，应当包含更详细的信息，并且随着代码的演进而持续更新。通过采取这些措施，可以提高代码的质量和团队的协作效率。 False
作为一个软件工程领域的工作者，我对SATD自承认技术债有一定的了解。在软件开发过程中，注释是一种重要的工具，用于记录代码的功能、意图以及存在的问题。对于注释`TODO: handle exception

2024-10-10 20:01:11 中国标准时间 - SparkPythonSDK - ERROR - [d:\dev\python3.8.10\lib\site-packages\sparkai\llm\llm.py:694] - SparkLLMClient wait LLM api response timeout 30 seconds


在软件工程中，注释是帮助开发者理解代码意图和设计决策的重要工具。你提到的注释 `TODO: a listener may be added many times if it implements many interfaces` 似乎是为了提醒或说明某个特定情况。然而，这个注释的表述可能不够清晰，因为它没有明确指出具体的实现细节、背景信息或者可能带来的影响。

为了更好地理解这段注释的目的和含义，我们可以从以下几个方面进行分析：

1. **上下文**：注释通常需要结合上下文来理解。我们需要知道这段代码是在什么情况下写的，是为了解决什么问题，或者是为了防止什么潜在的问题。

2. **目的**：注释可能是为了提醒开发者注意某个特定的行为或设计决策。在这个例子中，注释似乎是在说明一个监听器可以被多次添加，如果它实现了多个接口的话。这可能意味着在某个地方有一个逻辑是允许同一个对象被多次注册为监听器，只要它满足不同的接口要求。

3. **潜在问题**：这样的设计可能会导致一些问题，比如资源浪费、性能下降、难以追踪的行为等。注释可能是在提示开发者要注意这些潜在问题，并在后续的开发中考虑如何优化。

4. **改进建议**：为了让注释更加清晰，可以提供更详细的信息或者具体的改进建议。例如，可以解释为什么会有这种情况发生，以及如何避免或处理这种重复添加的问题。

综上所述，虽然这段注释提供了一个关于监听器可能被多次添加的提示，但其具体含义并不完全清晰。为了提高代码的可维护性和可读性，建议提供更详细的上下文信息和具体的改进建议。 False
作为软件工程领域的工作者，我认为注释"FIXME this is ugly hack!! it seems that dependency names from jboss-beans.xml no longer work in AS7!?"的目的含义是相对清晰的。

该注释明确指出代码中的某个部分存在一个“丑陋的”解决方案，即开发人员在处理与JBoss相关的依赖关系时遇到了问题，并且不得不使用了一个临时性、可能不理想的方法来解决。这种注释通常是自承认技术债（SATD）的一种形式，用于提醒其他开发者或未来的自己，此处需要进一步改进。 False
注释 `TODO: Consider handling the error differentl

2024-10-10 20:17:47 中国标准时间 - SparkPythonSDK - ERROR - [d:\dev\python3.8.10\lib\site-packages\sparkai\llm\llm.py:694] - SparkLLMClient wait LLM api response timeout 30 seconds


SparkAIConnectionError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。

对 sample_0.csv 和 sample_1.csv 中的 isClear 列进行显著性统计

看看 survive_time 为 0 和 1 的两组样本，它们的技术债务注释清晰度的比例是否有显著差异。

由于 isClear 是一个二元变量（True 或 False），我们可以使用 **卡方检验** 来进行显著性统计。

In [ ]:
from scipy.stats import chi2_contingency

# 读取csv文件
df_0 = pd.read_csv('sample_0.csv')  # 将'sample_0_with_isClear.csv'替换为你的文件名
df_1 = pd.read_csv('sample_1.csv')  # 将'sample_1_with_isClear.csv'替换为你的文件名

# 统计 isClear 的数量
count_0 = df_0['isClear'].value_counts()
count_1 = df_1['isClear'].value_counts()

# 构建列联表
contingency_table = pd.DataFrame({
    'survive_time_0': count_0,
    'survive_time_1': count_1
}).fillna(0)  # 用0填充缺失值

# 进行卡方检验
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# 打印结果
print('卡方检验结果:')
print(f'卡方值: {chi2}')
print(f'p值: {p_value}')
print(f'自由度: {dof}')
print('期望频数:\n', expected)

# 解释p值
alpha = 0.05  # 设置显著性水平
if p_value < alpha:
    print('拒绝原假设，两组样本的 isClear 比例存在显著差异。')
else:
    print('无法拒绝原假设，两组样本的 isClear 比例不存在显著差异。')

### Commit Workload：修复这一条SATD的作者在此前一个月的提交数量，检验

先获取这一次提交的作者的信息
通过特定提交获取提交作者的GitHub用户名： 使用 GitHub API 获取某次提交的详细信息。每个提交都会有一个 author 字段，包含提交人的 GitHub 用户名

In [ ]:
import requests

# GitHub 仓库和提交 SHA（提交的唯一标识符）
owner = "仓库所有者"
repo = "仓库名称"
commit_sha = "提交SHA"

url = f"https://api.github.com/repos/{owner}/{repo}/commits/{commit_sha}"

headers = {"Authorization": "token YOUR_GITHUB_TOKEN"}  # 可选：添加你的 GitHub Token
response = requests.get(url, headers=headers)

if response.status_code == 200:
    commit_data = response.json()
    author_login = commit_data["author"]["login"]
    print(f"Author's GitHub username: {author_login}")
else:
    print(f"Failed to get commit details: {response.status_code}")


获取该用户的提交记录： 一旦获取了该用户的 GitHub 用户名，你可以使用 GitHub 的 /repos/{owner}/{repo}/commits 端点来获取该用户的提交记录。

In [ ]:
def get_user_commits(owner, repo, author_login):
    url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    params = {"author": author_login}
    headers = {"Authorization": "token YOUR_GITHUB_TOKEN"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        commits = response.json()
        for commit in commits:
            print(f"Commit SHA: {commit['sha']}, Message: {commit['commit']['message']}")
    else:
        print(f"Failed to get commits: {response.status_code}")

# 调用函数获取该用户的提交记录
get_user_commits(owner, repo, author_login)
